In [ ]:
#The import cell: You'll only need to run this cell once, but you'll need to run it first
# To run, hold shift and press Enter.
import hoomd
import hoomd.md
from hoomd import deprecated
import mbuild, mdtraj, nglview, numpy
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# The 'settings' cell
# You can change these values to change how the simulations run
Number_of_particles = 100 #A quirk of this notebook is that you should choose a square number here
Temperature = 1.4 #These are "dimensionless temperature units" that map to Kelvin based upon the Lennard-Jones parameters

In [ ]:
# The simulation engine initializer cell
# This if you change something in the settings above, you'll need to re-run that cell and this one before 
# running another simulation
hoomd.context.initialize("")
system=hoomd.init.create_lattice(unitcell=hoomd.lattice.unitcell(N=1,a1=[1.1,0,0],a2=[0,1.1,0],a3=[0,0,1],dimensions=2,type_name=["A"],position=[[0,0,0]]),n=int(Number_of_particles**.5))
deprecated.dump.xml(filename="init.hoomdxml",group=hoomd.group.all(),vis=True)
nl = hoomd.md.nlist.tree()
lj = hoomd.md.pair.lj(r_cut=2.5,nlist=nl)
lj.pair_coeff.set("A","A",epsilon=1,sigma=1)
hoomd.md.integrate.mode_standard(dt=0.007)
all=hoomd.group.all()
hoomd.md.integrate.langevin(group=all,kT=Temperature,seed=14)

In [ ]:
#The simulation cell
# Run this cell to output a 'log' file, plus a couple of files that store info for all the microstates sampled
hoomd.dump.dcd(filename="traj.dcd",period=50,group=hoomd.group.all(),overwrite=True)
hoomd.analyze.log(filename="log",quantities=['potential_energy','temperature','pressure','kinetic_energy'],period=50,overwrite=True)
hoomd.run(5e4)
deprecated.dump.xml(filename="final.hoomdxml",group=hoomd.group.all(),vis=True)

In [ ]:
# The visualization cell
# Run this to see the atoms move!
t = mdtraj.load("traj.dcd",top="final.hoomdxml")
view = nglview.show_mdtraj(t)
view.clear_representations()
view.add_representation("point",selection=[i for i in range(0,Number_of_particles//2)],color='blue',pointSize=2.5,useTexture=True)
view.add_representation("point",selection=[i for i in range(Number_of_particles//2,Number_of_particles)],color='orange',pointSize=2.5,useTexture=True)
view
#After running this cell, you can press "play" below and use the slider to view different microstates.
#Sometimes you'll get a bunch of pink warnings if the slider starts over - you can ignore these safely.

In [ ]:
#The data in the log file has the timestep in column 0, PE in column 1, T in column 2, P in column 3, and KE in column 4
data=numpy.loadtxt("log",skiprows=1)
plt.plot(data[:,1]) #Remember this excellent slice notation from MSE 150? Hooray!!
plt.ylabel("Potential Energy")
print("Average PE: {}".format(numpy.mean(data[:,1])))

In [ ]:
plt.plot(data[:,2])
plt.ylabel("Temperature")

In [ ]:
plt.plot(data[:,3])
plt.ylabel("Pressure")

In [ ]:
# What's the pressure when T=1.0?

In [ ]:
# What's the probability of observing the total system energy outside 1 standard deviation at T=1.0?
# a) Calculate the total energy of the system as a function of time (Potential plus kinetic energy)
Total_Energy = data[:,1] + data[:,FILL_IN_THIS_BLANK]
# b) Calculate the average and standard deviation of total energy
average = numpy.mean(Total_Energy)
standard_deviation = numpy.std(Total_Energy)
# c) find the entries outside this range
high_anomalies = Total_Energy[Total_Energy>average+standard_deviation] # You can ignore how this logic works for now, or come chat if you're curious! But it works.
low_anomalies = Total_Energy[Total_Energy<average-FILL_IN_THIS_BLANK]
print("prob:{}".format((len(high_anomalies)+len(low_anomalies))/len(FILL_IN_THIS_BLANK)))